In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import LabelEncoder, RobustScaler
c_products=pd.read_csv('cproducts.csv')


In [3]:
c_products.head()

,customerID,DOB,Gender,State,PinCode,transactionDate,store_code,store_description,till_no,transaction_number_by_till,promo_code,promotion_description,product_code,product_description,sale_price_after_promo,discountUsed
0,BBID_20482,1975-10-20,male,MADHYA PRADESH,453441.0,2016-05-01,2655,BB-INDORE-MALHAR MEGA MALL,2,85246,NONPROMO,NaN,1000446431,PATANJALI CHOCOS 125g,55.00,Payback
1,BBID_20485,1955-07-21,female,NaN,999999.0,2017-03-09,2655,BB-INDORE-MALHAR MEGA MALL,8,78829,NONPROMO,NaN,1000010431,SOO FRESH ROLLS HOT DOG 4P,30.00,Payback
2,BBID_20498,1981-03-19,female,MAHARASHTRA,400064.0,2015-08-12,2655,BB-INDORE-MALHAR MEGA MALL,13,1064,0000971754,AD 399 FLAT LADIES JEANS FASHION EXC,1000077851005,"DLJ-0010-BSF-2-CJ-FL, 34, BLACK",418.95,Payback
3,BBID_20499,1986-05-10,male,MAHARASHTRA,400099.0,2016-12-18,2615,BB-AMRITSAR-TRILIUM MALL,14,51080,NONPROMO,NaN,1000443231,PATANJALI TOMATO KETCHUP BT 500g,70.00,Payback
4,BBID_204110,1988-02-29,male,MADHYA PRADESH,474001.0,2015-12-31,2655,BB-INDORE-MALHAR MEGA MALL,9,21911,0000974126,NMP 5% OFF CONCEPT DISCOUNT 2655,300481741,MAGGI NDL MERI MASALA PP 70g,34.20,Payback


In [4]:
c_products.DOB=c_products.DOB.replace('NANA',"2000-01-01")
c_products.DOB.fillna("2001-01-01",inplace=True)


In [5]:
import datetime as dt

d = [dt.datetime.strptime(str(s), "%Y-%m-%d") for s in c_products.DOB]

In [6]:
t = [dt.datetime.strptime(str(td), "%Y-%m-%d") for td in c_products.transactionDate]

In [7]:
obj_df = c_products.select_dtypes(include=['object']).copy()
obj_df.head()

,customerID,DOB,Gender,State,transactionDate,store_description,promo_code,promotion_description,product_description,discountUsed
0,BBID_20482,1975-10-20,male,MADHYA PRADESH,2016-05-01,BB-INDORE-MALHAR MEGA MALL,NONPROMO,NaN,PATANJALI CHOCOS 125g,Payback
1,BBID_20485,1955-07-21,female,NaN,2017-03-09,BB-INDORE-MALHAR MEGA MALL,NONPROMO,NaN,SOO FRESH ROLLS HOT DOG 4P,Payback
2,BBID_20498,1981-03-19,female,MAHARASHTRA,2015-08-12,BB-INDORE-MALHAR MEGA MALL,0000971754,AD 399 FLAT LADIES JEANS FASHION EXC,"DLJ-0010-BSF-2-CJ-FL, 34, BLACK",Payback
3,BBID_20499,1986-05-10,male,MAHARASHTRA,2016-12-18,BB-AMRITSAR-TRILIUM MALL,NONPROMO,NaN,PATANJALI TOMATO KETCHUP BT 500g,Payback
4,BBID_204110,1988-02-29,male,MADHYA PRADESH,2015-12-31,BB-INDORE-MALHAR MEGA MALL,0000974126,NMP 5% OFF CONCEPT DISCOUNT 2655,MAGGI NDL MERI MASALA PP 70g,Payback


In [8]:
c_products=c_products.drop(['DOB'],axis=1)
c_products=c_products.drop(['transactionDate'], axis=1)

In [9]:
c_products['day']=[x.day for x in d]

In [10]:
c_products['month']=[x.month for x in d]

In [11]:
c_products['t_day']=[x.day for x in t]
c_products['t_month']=[x.month for x in t]

In [12]:
c_products.head()

,customerID,Gender,State,PinCode,store_code,store_description,till_no,transaction_number_by_till,promo_code,promotion_description,product_code,product_description,sale_price_after_promo,discountUsed,day,month,t_day,t_month
0,BBID_20482,male,MADHYA PRADESH,453441.0,2655,BB-INDORE-MALHAR MEGA MALL,2,85246,NONPROMO,NaN,1000446431,PATANJALI CHOCOS 125g,55.00,Payback,20,10,1,5
1,BBID_20485,female,NaN,999999.0,2655,BB-INDORE-MALHAR MEGA MALL,8,78829,NONPROMO,NaN,1000010431,SOO FRESH ROLLS HOT DOG 4P,30.00,Payback,21,7,9,3
2,BBID_20498,female,MAHARASHTRA,400064.0,2655,BB-INDORE-MALHAR MEGA MALL,13,1064,0000971754,AD 399 FLAT LADIES JEANS FASHION EXC,1000077851005,"DLJ-0010-BSF-2-CJ-FL, 34, BLACK",418.95,Payback,19,3,12,8
3,BBID_20499,male,MAHARASHTRA,400099.0,2615,BB-AMRITSAR-TRILIUM MALL,14,51080,NONPROMO,NaN,1000443231,PATANJALI TOMATO KETCHUP BT 500g,70.00,Payback,10,5,18,12
4,BBID_204110,male,MADHYA PRADESH,474001.0,2655,BB-INDORE-MALHAR MEGA MALL,9,21911,0000974126,NMP 5% OFF CONCEPT DISCOUNT 2655,300481741,MAGGI NDL MERI MASALA PP 70g,34.20,Payback,29,2,31,12


In [13]:
obj_df = c_products.select_dtypes(include=['object']).copy()
obj_df.head()

,customerID,Gender,State,store_description,promo_code,promotion_description,product_description,discountUsed
0,BBID_20482,male,MADHYA PRADESH,BB-INDORE-MALHAR MEGA MALL,NONPROMO,NaN,PATANJALI CHOCOS 125g,Payback
1,BBID_20485,female,NaN,BB-INDORE-MALHAR MEGA MALL,NONPROMO,NaN,SOO FRESH ROLLS HOT DOG 4P,Payback
2,BBID_20498,female,MAHARASHTRA,BB-INDORE-MALHAR MEGA MALL,0000971754,AD 399 FLAT LADIES JEANS FASHION EXC,"DLJ-0010-BSF-2-CJ-FL, 34, BLACK",Payback
3,BBID_20499,male,MAHARASHTRA,BB-AMRITSAR-TRILIUM MALL,NONPROMO,NaN,PATANJALI TOMATO KETCHUP BT 500g,Payback
4,BBID_204110,male,MADHYA PRADESH,BB-INDORE-MALHAR MEGA MALL,0000974126,NMP 5% OFF CONCEPT DISCOUNT 2655,MAGGI NDL MERI MASALA PP 70g,Payback


In [14]:
for col in obj_df.columns:
    c_products[col].fillna(c_products[col].mode()[0], inplace=True)

In [15]:
c_products.head()
c_products['promotion_description'].fillna('no_promo', inplace=True)
c_products['Gender'].fillna('no_gender', inplace=True)
c_products['State'].fillna('no_state', inplace=True)
c_products['PinCode'].fillna(-1, inplace=True)

In [16]:
from sklearn import preprocessing

cols=['Gender','State','store_description','promo_code','promotion_description','product_description','discountUsed']
for col in cols:
    le = preprocessing.LabelEncoder()
    c_products[col]=le.fit_transform(c_products[col])
    

In [17]:
c_products.head()

,customerID,Gender,State,PinCode,store_code,store_description,till_no,transaction_number_by_till,promo_code,promotion_description,product_code,product_description,sale_price_after_promo,discountUsed,day,month,t_day,t_month
0,BBID_20482,1,20,453441.0,2655,2,2,85246,2213,72,1000446431,4269,55.00,24,20,10,1,5
1,BBID_20485,0,20,999999.0,2655,2,8,78829,2213,72,1000010431,5121,30.00,24,21,7,9,3
2,BBID_20498,0,21,400064.0,2655,2,13,1064,216,23,1000077851005,1617,418.95,24,19,3,12,8
3,BBID_20499,1,21,400099.0,2615,0,14,51080,2213,72,1000443231,4364,70.00,24,10,5,18,12
4,BBID_204110,1,20,474001.0,2655,2,9,21911,225,1723,300481741,3472,34.20,24,29,2,31,12


In [18]:
#adding column date,month,year of D.O.B
c_products['day']=[x.day for x in d]
c_products['month']=[x.month for x in d]


#adding  column t_date,t_month,t_year of transactionalDate
c_products['t_day']=[x.day for x in t]
c_products['t_month']=[x.month for x in t]


In [54]:
## scaling, creating matrix and running k-means


stores = list(set(c_products['store_code']))

cluster_labels = []
cluster_store = []
cluster_data = []
cluster_customers = []
cluster_score = []

for x in stores:
    cld = c_products[c_products['store_code'] == x]
    cluster_customers.append(cld['customerID'])
    #cld.drop(['customerID','promotion_description','product_code'], axis=1, inplace=True)#0.79726,0.7914,n=4
    #cld.drop(['customerID','product_code','store_description'], axis=1, inplace=True)#0.76653 ,n=5
    #cld.drop(['store_code','customerID','t_day', 'store_description'], axis=1, inplace=True)
    #cld.drop(['customerID','promotion_description','product_code','discountUsed'], axis=1, inplace=True)
    #cld.drop(['store_code','customerID','product_code'], axis=1, inplace=True)#0.77195
    
     
    cld.drop(['store_code','customerID','product_code'], axis=1, inplace=True)#0.81205,n=5
   
    rbs = RobustScaler()
    cld2 = rbs.fit_transform(cld)
   

    km1 = KMeans(n_clusters=5)
    km2 = km1.fit(cld2)
    label = km2.predict(cld2) #af.labels_ 
    
    s_score = silhouette_score(cld2, label)
    cluster_score.append(s_score)
    
    cluster_labels.append(label)
    cluster_store.append(np.repeat(x, cld.shape[0]))
    cluster_data.append(cld2)


C:\Users\SIDDHARTH\Anaconda3\lib\site-packages\ipykernel\__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [55]:
np.mean(cluster_score)
s_score

0.92783359967014378

In [56]:
cluster_data = np.concatenate(cluster_data)

In [57]:
cluster_data.shape

(11972, 15)

In [58]:

## convert nested lists as 1d array
cluster_customers = np.concatenate(cluster_customers)
cluster_store = np.concatenate(cluster_store)
cluster_labels = np.concatenate(cluster_labels)

In [59]:
## create submission files
sub1 = pd.DataFrame({'customerID':cluster_customers, 'store_code':cluster_store, 'cluster':cluster_labels})

In [60]:
np.savetxt('subOne_18.txt', cluster_data)
sub1.to_csv('subtwo_18.csv', index=False)

In [61]:
file = open("cluster.txt", "a")
for i in range(len(Z)):
    file.write(str(Z[i]))
file.close()

NameError: name 'Z' is not defined

In [ ]:
np.savetxt('sub.txt',np.concatenate(Z))

In [ ]:
submission=pd.DataFrame()
submission['customerID']=c_products['customerID']
submission['store_code']=obj_df.store_code
submission['cluster']=clusters
submission.to_csv('submission.csv',index=False)

In [ ]:
obj_df.head()